# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)


## 1. Data Wrangling and Cleaning

### 1.1 Portfolio dataframe

In [2]:
#Check the first entries
print(portfolio.head())
#Check the data types
print(portfolio.dtypes)

                       channels  difficulty  duration  \
0       [email, mobile, social]          10         7   
1  [web, email, mobile, social]          10         5   
2          [web, email, mobile]           0         4   
3          [web, email, mobile]           5         7   
4                  [web, email]          20        10   

                                 id     offer_type  reward  
0  ae264e3637204a6fb9bb56bc8210ddfd           bogo      10  
1  4d5c57ea9a6940dd891ad53e9dbe8da0           bogo      10  
2  3f207df678b143eea3cee63160fa8bed  informational       0  
3  9b98b8c7a33c4b65b9aebfe6a799e6d9           bogo       5  
4  0b1e1539f2cc45b7b9fa7c272da2e1d7       discount       5  
channels      object
difficulty     int64
duration       int64
id            object
offer_type    object
reward         int64
dtype: object


In [3]:
# Split channels and offer_type into dummy variables
dummies = pd.get_dummies(portfolio['offer_type'])
print(dummies)

#Concat dummy variables to dataframe
portfolio = pd.concat([portfolio,dummies],axis=1)
#portfolio = portfolio.drop('offer_type',axis=1)

#Check if everything went well
portfolio.head()

   bogo  discount  informational
0     1         0              0
1     1         0              0
2     0         0              1
3     1         0              0
4     0         1              0
5     0         1              0
6     0         1              0
7     0         0              1
8     1         0              0
9     0         1              0


,channels,difficulty,duration,id,offer_type,reward,bogo,discount,informational
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,1,0,0
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,1,0,0
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0,0,0,1
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,1,0,0
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5,0,1,0


In [4]:
#Create dummy variables out of "channels" column of lists

#Create column for every value in list in the "channels" column and et the value 1 for it.
for i in range(len(portfolio.channels)):
    for j in portfolio.loc[i,"channels"]:
        portfolio.loc[i,j]=1 

#Short check
portfolio.head()

#Fill nan values with 0 
portfolio.email=portfolio.email.fillna(0)
portfolio.mobile=portfolio.mobile.fillna(0)
portfolio.social=portfolio.social.fillna(0)
portfolio.web=portfolio.web.fillna(0)

#Short check again
portfolio.head()

#Drop the "channels" column
portfolio.drop('channels',axis=1)

,difficulty,duration,id,offer_type,reward,bogo,discount,informational,email,mobile,social,web
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,1,0,0,1.0,1.0,1.0,0.0
1,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,1,0,0,1.0,1.0,1.0,1.0
2,0,4,3f207df678b143eea3cee63160fa8bed,informational,0,0,0,1,1.0,1.0,0.0,1.0
3,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,1,0,0,1.0,1.0,0.0,1.0
4,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5,0,1,0,1.0,0.0,0.0,1.0
5,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3,0,1,0,1.0,1.0,1.0,1.0
6,10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2,0,1,0,1.0,1.0,1.0,1.0
7,0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0,0,0,1,1.0,1.0,1.0,0.0
8,5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,1,0,0,1.0,1.0,1.0,1.0
9,10,7,2906b810c7d4411798c6938adc9daaa5,discount,2,0,1,0,1.0,1.0,0.0,1.0


In [5]:
#Rename ID-column into "offer_id"
portfolio = portfolio.rename(columns={"id":"offer_id"})
portfolio.head()

,channels,difficulty,duration,offer_id,offer_type,reward,bogo,discount,informational,email,mobile,social,web
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,1,0,0,1.0,1.0,1.0,0.0
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,1,0,0,1.0,1.0,1.0,1.0
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0,0,0,1,1.0,1.0,0.0,1.0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,1,0,0,1.0,1.0,0.0,1.0
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5,0,1,0,1.0,0.0,0.0,1.0


### 1.2 Profile dataframe 

In [6]:
#Check the first entries
profile.head()

#Check the data types
profile.dtypes

age                   int64
became_member_on      int64
gender               object
id                   object
income              float64
dtype: object

In [7]:
# Clean mistakes with age 118. Change it to 999.
c = 0
for i in profile.age:
    if i == 118:
        profile.loc[c,'age'] = 999
    c+=1
profile.head()

,age,became_member_on,gender,id,income
0,999,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,999,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,999,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [8]:
#Change format of the date
profile.became_member_on=profile.became_member_on.astype(str)
profile.loc[:,'became_member_on'] = profile.loc[:,'became_member_on'].str[:4]+'.'+profile.loc[:,'became_member_on'].str[4:6]+'.'+profile.loc[:,'became_member_on'].str[6:]
profile.head()

,age,became_member_on,gender,id,income
0,999,2017.02.12,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,2017.07.15,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,999,2018.07.12,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,2017.05.09,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,999,2017.08.04,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [9]:
#Fill up nan values with 0 for "income"
profile.income = profile.income.fillna(0)
profile.age = profile.age.fillna("None")
profile.head()

,age,became_member_on,gender,id,income
0,999,2017.02.12,None,68be06ca386d4c31939f3a4f0e3dd783,0.0
1,55,2017.07.15,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,999,2018.07.12,None,38fe809add3b4fcf9315a9694bb96ff5,0.0
3,75,2017.05.09,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,999,2017.08.04,None,a03223e636434f42ac4c3df47e8bac43,0.0


In [10]:
#Rename ID-column in "customer_id"
profile = profile.rename(columns={"id":"customer_id"})
profile.head()

,age,became_member_on,gender,customer_id,income
0,999,2017.02.12,None,68be06ca386d4c31939f3a4f0e3dd783,0.0
1,55,2017.07.15,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,999,2018.07.12,None,38fe809add3b4fcf9315a9694bb96ff5,0.0
3,75,2017.05.09,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,999,2017.08.04,None,a03223e636434f42ac4c3df47e8bac43,0.0


### 1.3 Transcript dataframe

In [11]:
#Check the first entries
print(transcript.head())

#Check the data types
print(transcript.dtypes)

            event                            person  time  \
0  offer received  78afa995795e4d85b5d9ceeca43f5fef     0   
1  offer received  a03223e636434f42ac4c3df47e8bac43     0   
2  offer received  e2127556f4f64592b11af22de27a7932     0   
3  offer received  8ec6ce2a7e7949b1bf142def7d0e0586     0   
4  offer received  68617ca6246f4fbc85e91a2a49552598     0   

                                              value  
0  {'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}  
1  {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}  
2  {'offer id': '2906b810c7d4411798c6938adc9daaa5'}  
3  {'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}  
4  {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}  
event     object
person    object
time       int64
value     object
dtype: object


In [12]:
#Create dummy variables out of "event" column
dummies = pd.get_dummies(transcript.event)
print(dummies)

#Concat the dummies with the dataframe
transcript = pd.concat([transcript,dummies],axis=1)
transcript = transcript.drop('event',axis=1)
transcript.head()

        offer completed  offer received  offer viewed  transaction
0                     0               1             0            0
1                     0               1             0            0
2                     0               1             0            0
3                     0               1             0            0
4                     0               1             0            0
5                     0               1             0            0
6                     0               1             0            0
7                     0               1             0            0
8                     0               1             0            0
9                     0               1             0            0
10                    0               1             0            0
11                    0               1             0            0
12                    0               1             0            0
13                    0               1             0         

,person,time,value,offer completed,offer received,offer viewed,transaction
0,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,1,0,0
1,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,1,0,0
2,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,1,0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,1,0,0
4,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,1,0,0


In [13]:
#Rename ID-column to "customer_id"
transcript = transcript.rename(columns={"person":"customer_id"})
transcript.head()

,customer_id,time,value,offer completed,offer received,offer viewed,transaction
0,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,1,0,0
1,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,1,0,0
2,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,1,0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,1,0,0
4,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,1,0,0


In [14]:
# Create new columns with the keys of the "value" column and fill in the values of the keys
a = transcript["value"].apply(pd.Series)

#Concat the new key columns with the dataframe 
transcript_neo = pd.concat([transcript,a],axis=1)

# Drop extra column ("offer_id")
transcript_neo = transcript_neo.drop(["offer_id"],axis=1)

#Rename the "offer id" column into "offer_id"
transcript_neo = transcript_neo.rename(columns={"offer id":"offer_id"})

#Fill in the nan values
transcript_neo["reward"] = transcript_neo["reward"].fillna(0)
transcript_neo["offer_id"]=transcript_neo["offer_id"].fillna("None")
transcript_neo["amount"]=transcript_neo["amount"].fillna(0)

#Drop the original value-column
transcript_neo = transcript_neo.drop(["value"],axis=1)
transcript_neo.head()

,customer_id,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0,0.0
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,0.0,0.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,0.0,0.0
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0.0


In [15]:
transcript_neo.columns

Index(['customer_id', 'time', 'offer completed', 'offer received',
       'offer viewed', 'transaction', 'offer_id', 'amount', 'reward'],
      dtype='object')

### 1.4 Merge everything to start the analysis

In [16]:
# Merge the three dataframes 
df = profile.merge(transcript_neo,how='left',on='customer_id')
df = df.merge(portfolio,how='left',on='offer_id')
df.head()

,age,became_member_on,gender,customer_id,income,time,offer completed,offer received,offer viewed,transaction,...,duration,offer_type,reward_y,bogo,discount,informational,email,mobile,social,web
0,999,2017.02.12,None,68be06ca386d4c31939f3a4f0e3dd783,0.0,168,0,1,0,0,...,7.0,discount,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
1,999,2017.02.12,None,68be06ca386d4c31939f3a4f0e3dd783,0.0,216,0,0,1,0,...,7.0,discount,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
2,999,2017.02.12,None,68be06ca386d4c31939f3a4f0e3dd783,0.0,336,0,1,0,0,...,10.0,discount,5.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,999,2017.02.12,None,68be06ca386d4c31939f3a4f0e3dd783,0.0,348,0,0,1,0,...,10.0,discount,5.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,999,2017.02.12,None,68be06ca386d4c31939f3a4f0e3dd783,0.0,360,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Fill every NaN values with values from one row above, to completely analyse the dataframe

In [17]:
df_neo = pd.DataFrame()

# Get rid of the NaN values in the dataframe, which are a result of the merging with missing offer_ids due to transaction values
def fill_na_in_column(df,column):
    c = 0
    while df[column].isnull().sum() > 0:
        #print("This is loop "+str(c+1))
        for i in range(df.shape[0]):
            #print(i)
            if i == 0:
                #print("nicht nan")
                df.loc[i,column] = df.loc[i,column]
            else:
                if pd.isnull(df.loc[i,column]):
                    #print("nan")
                    df.loc[i,column] = df.loc[i-1,column]
                else:
                    #print("nicht nan")
                    df.loc[i,column] = df.loc[i,column]
        c += 1
    return df[column]

columns = df.columns
for column in columns:
    df_neo[column] = fill_na_in_column(df,column)

df_neo.to_pickle("merged_df.pkl") 

df_neo.head(100)

## Exploration

In [18]:
# Number of customers
print(df['customer_id'].nunique())
# Number of data entries for alle customers
print(df.shape[0])
# Number of entries per customer on average
print(df.shape[0]/df['customer_id'].nunique())

17000
306534
18.031411764705883


In [31]:
df.head(100)

,age,became_member_on,gender,customer_id,income,time,offer completed,offer received,offer viewed,transaction,...,duration,offer_type,reward_y,bogo,discount,informational,email,mobile,social,web
0,999,2017.02.12,None,68be06ca386d4c31939f3a4f0e3dd783,0.0,168,0,1,0,0,...,7.0,discount,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
1,999,2017.02.12,None,68be06ca386d4c31939f3a4f0e3dd783,0.0,216,0,0,1,0,...,7.0,discount,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
2,999,2017.02.12,None,68be06ca386d4c31939f3a4f0e3dd783,0.0,336,0,1,0,0,...,10.0,discount,5.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,999,2017.02.12,None,68be06ca386d4c31939f3a4f0e3dd783,0.0,348,0,0,1,0,...,10.0,discount,5.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,999,2017.02.12,None,68be06ca386d4c31939f3a4f0e3dd783,0.0,360,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,999,2017.02.12,None,68be06ca386d4c31939f3a4f0e3dd783,0.0,408,0,1,0,0,...,10.0,discount,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
6,999,2017.02.12,None,68be06ca386d4c31939f3a4f0e3dd783,0.0,408,0,0,1,0,...,10.0,discount,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
7,999,2017.02.12,None,68be06ca386d4c31939f3a4f0e3dd783,0.0,414,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,999,2017.02.12,None,68be06ca386d4c31939f3a4f0e3dd783,0.0,444,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,999,2017.02.12,None,68be06ca386d4c31939f3a4f0e3dd783,0.0,504,0,1,0,0,...,7.0,discount,3.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0


In [20]:
df.columns

Index(['age', 'became_member_on', 'gender', 'customer_id', 'income', 'time',
       'offer completed', 'offer received', 'offer viewed', 'transaction',
       'offer_id', 'amount', 'reward_x', 'channels', 'difficulty', 'duration',
       'offer_type', 'reward_y', 'bogo', 'discount', 'informational', 'email',
       'mobile', 'social', 'web'],
      dtype='object')

In [21]:
# Build customer statistics for further analysis
def build_customer_statistics(df,sum_num_columns,non_sum_num_columns):
    cust_sums = df.groupby(['customer_id']).sum()[sum_num_columns]
    cust_demos = df.groupby(['customer_id']).max()[non_sum_num_columns]
    custom_df = cust_sums.merge(cust_demos,how='left',on='customer_id')
    df_neo = custom_df
    return df_neo
    
sum_num_columns = ['offer received','offer viewed','transaction','offer completed','difficulty','amount','reward_y','bogo','discount','informational']
non_sum_num_columns = ['age','gender','income']

custom_df = build_customer_statistics(df,sum_num_columns,non_sum_num_columns)

In [22]:
custom_df.head()

,offer received,offer viewed,transaction,offer completed,difficulty,amount,reward_y,bogo,discount,informational,age,gender,income
customer_id,,,,,,,,,,,,,
0009655768c64bdeb2e877511632db8f,5,4,8,3,40.0,127.60,16.0,2.0,3.0,4.0,33,M,72000.0
00116118485d4dfda04fdbaba9a87b5c,2,2,3,0,20.0,4.09,20.0,4.0,0.0,0.0,999,NaN,0.0
0011e0d4e6b944f998e987f904e8c1e5,5,5,5,3,64.0,79.46,26.0,2.0,4.0,4.0,40,O,57000.0
0020c2b971eb4e9188eac86d93036a77,5,3,8,3,60.0,196.86,36.0,3.0,3.0,2.0,59,F,90000.0
0020ccbbb6d84e358d3414a3ff76cffd,4,4,12,3,34.0,154.05,26.0,4.0,2.0,2.0,24,F,60000.0


In [23]:
def generate_general_metrics(custom_df):
    # Generate general metrics over all offers

    # Number of received offers
    n_offers = np.sum(custom_df['offer received'])

    # View rate (Rate, that a customer viewed an offer)
    view_rate = np.sum(custom_df['offer viewed'])/np.sum(custom_df['offer received'])
    
    # Completion rate (Rate, that a customer completed an offer)
    comp_rate = np.sum(custom_df['offer completed'])/np.sum(custom_df['offer received'])
    
    # View-Completion rate (Rate, that a customer completed an offer after seeing it).
    comp_view_rate = np.sum(custom_df['offer completed'])/np.sum(custom_df['offer viewed'])
    
    # Transaction-Completion rate (Rate, that a customer completed an offer because of an transaction).
    comp_trans_rate = np.sum(custom_df['offer completed'])/np.sum(custom_df['transaction'])


    print("The number of received offers: "+str(n_offers*100)+" %")
    print("The rate of viewed offers: "+str(view_rate*100)+" %")
    print("The rate of completing offers: "+str(comp_rate*100)+" %")
    print("The rate of completing offers after seeing them: "+str(comp_view_rate*100)+" %")
    print("The rate of completing offers because of a transaction: "+str(comp_trans_rate*100)+" %")
    
generate_general_metrics(custom_df)

The number of received offers: 7627700 %
The rate of viewed offers: 75.6781205344 %
The rate of completing offers: 44.0224445115 %
The rate of completing offers after seeing them: 58.1706366392 %
The rate of completing offers because of a transaction: 24.1657251013 %


In [24]:
custom_df['view_rate']=custom_df['offer viewed']/custom_df['offer received']
custom_df['comp_rate']= custom_df['offer completed']/custom_df['offer received']
custom_df['comp_view_rate']=custom_df['offer completed']/custom_df['offer viewed']
custom_df.head()

,offer received,offer viewed,transaction,offer completed,difficulty,amount,reward_y,bogo,discount,informational,age,gender,income,view_rate,comp_rate,comp_view_rate
customer_id,,,,,,,,,,,,,,,,
0009655768c64bdeb2e877511632db8f,5,4,8,3,40.0,127.60,16.0,2.0,3.0,4.0,33,M,72000.0,0.8,0.60,0.75
00116118485d4dfda04fdbaba9a87b5c,2,2,3,0,20.0,4.09,20.0,4.0,0.0,0.0,999,NaN,0.0,1.0,0.00,0.00
0011e0d4e6b944f998e987f904e8c1e5,5,5,5,3,64.0,79.46,26.0,2.0,4.0,4.0,40,O,57000.0,1.0,0.60,0.60
0020c2b971eb4e9188eac86d93036a77,5,3,8,3,60.0,196.86,36.0,3.0,3.0,2.0,59,F,90000.0,0.6,0.60,1.00
0020ccbbb6d84e358d3414a3ff76cffd,4,4,12,3,34.0,154.05,26.0,4.0,2.0,2.0,24,F,60000.0,1.0,0.75,0.75


### We can say, that the probability of completing an offer after viewing it is two times higher than the probability to complete an offer by chance (by transactions without knowledge) 

### We have to wrangle the merged dataframe a bit to split it into the offer_types and analyse it.

In [25]:
#Now split the offers by offer type and check for same metrics

# Create subsets of the types of discounts the customer got
bogo = df[df['bogo']== 1]
informational = df[df['informational']==1]
discount = df[df['discount'] == 1]

In [26]:
#Build customer statstics for BOGO, discount and informational offer type
customer_bogo = build_customer_statistics(bogo,sum_num_columns,non_sum_num_columns)
generate_general_metrics(customer_bogo)

customer_informational = build_customer_statistics(informational,sum_num_columns,non_sum_num_columns)
generate_general_metrics(customer_informational)

customer_discount = build_customer_statistics(discount,sum_num_columns,non_sum_num_columns)
generate_general_metrics(customer_informational)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in long_scalars


The number of received offers: 3049900 %
The rate of viewed offers: 83.4420800682 %
The rate of completing offers: 0.0 %
The rate of completing offers after seeing them: 0.0 %
The rate of completing offers because of a transaction: nan %
The number of received offers: 1523500 %
The rate of viewed offers: 71.0928782409 %
The rate of completing offers: 0.0 %
The rate of completing offers after seeing them: 0.0 %
The rate of completing offers because of a transaction: nan %
The number of received offers: 1523500 %
The rate of viewed offers: 71.0928782409 %
The rate of completing offers: 0.0 %
The rate of completing offers after seeing them: 0.0 %
The rate of completing offers because of a transaction: nan %


In [27]:
print(bogo.head())
print(customer_bogo.head())

    age became_member_on gender                       customer_id    income  \
23   55       2017.07.15      F  0610b486422d4921ae7d2bf64640c50b  112000.0   
28  999       2018.07.12   None  38fe809add3b4fcf9315a9694bb96ff5       0.0   
29  999       2018.07.12   None  38fe809add3b4fcf9315a9694bb96ff5       0.0   
37   75       2017.05.09      F  78afa995795e4d85b5d9ceeca43f5fef  100000.0   
38   75       2017.05.09      F  78afa995795e4d85b5d9ceeca43f5fef  100000.0   

    time  offer completed  offer received  offer viewed  transaction ...   \
23   408                0               1             0            0 ...    
28   168                0               1             0            0 ...    
29   168                0               0             1            0 ...    
37     0                0               1             0            0 ...    
38     6                0               0             1            0 ...    

   duration  offer_type  reward_y bogo  discount  informationa

In [32]:
# This is just a test to see the data structure
test = df[df['customer_id']=='38fe809add3b4fcf9315a9694bb96ff5'][['time','offer_type','offer received','offer viewed', 'transaction','offer completed']]
test

,time,offer_type,offer received,offer viewed,transaction,offer completed
27,132,NaN,0,0,1,0
28,168,bogo,1,0,0,0
29,168,bogo,0,1,0,0
30,348,NaN,0,0,1,0
31,450,NaN,0,0,1,0
32,474,NaN,0,0,1,0
33,576,informational,1,0,0,0
34,636,NaN,0,0,1,0
35,666,informational,0,1,0,0
36,696,NaN,0,0,1,0


### Resources
- https://stackoverflow.com/questions/5254445/add-string-in-a-certain-position-in-python  
- https://stackoverflow.com/questions/38231591/split-explode-a-column-of-dictionaries-into-separate-columns-with-pandas  
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html  
